## Data discovery: Load and query Yellow Taxi data
> Download the dataset from [the official TLC Trip Record Data website](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

---

### This cell only shows how to document code
```python
# Load file
local_file = 'datasets/your-downloaded-from-TLC-taxis-file-here.parquet'

# Show data
spark.read.parquet(local_file).show()
```

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col

### What is master(local N)?
The --master option specifies the master URL for a distributed cluster, or local to run locally with one thread, or local[N] to run locally with N threads.

<b>Source</b>: See Spark [docs here](spark.apache.org/docs/latest). See all [options here](https://spark.apache.org/docs/latest/submitting-applications.html#master-urls)

In [ ]:
# Create SparkSession
spark = SparkSession.builder\
             .master("local[1]")\
             .appName("spark-app-version-x")\
             .getOrCreate()

In [ ]:
# Read taxi data
local_files = 'datasets/parquet/'
df = spark.read.parquet(local_files)

In [ ]:
# DF is like a relation table in memory. Let's see the columns
df.printSchema()

In [ ]:
# Query sample:
df.select('VendorID','total_amount', 'PULocationID').show(n=5)

In [ ]:
# Query sample, using Spark SQL
df.createOrReplaceTempView('tbl_raw_yellow_taxis')

In [ ]:
# SQL Statement
# PULocationID = 188, 379 rows our of 3,066,766
spark.sql('''
          select min(tpep_pickup_datetime), max(tpep_dropoff_datetime)
          from tbl_raw_yellow_taxis
          ''').show(n=5)

In [ ]:
# SQL Statement
spark.sql('''
          select extract(year from tpep_pickup_datetime), count(1)
          from tbl_raw_yellow_taxis
          group by extract(year from tpep_pickup_datetime)
          having count(1) > 100
          ''').show(n=100)

In [ ]:
# SQL Statement example, using a subquery to clean the data
# Use case example: imagine our business users asked to us delete all data if dataset's year has < 100 rows.
df_clean_s1 = spark.sql('''
          select *
          from tbl_raw_yellow_taxis
          where extract(year from tpep_pickup_datetime) in
                        (select extract(year from tpep_pickup_datetime)
                        from tbl_raw_yellow_taxis
                        group by extract(year from tpep_pickup_datetime)
                        having count(1) > 100
                        )
          ''')

In [ ]:
# Register new Temp View, using the cleansed new DataFrame 
df_clean_s1.createOrReplaceTempView('tbl_raw_yellow_taxis_clean_s1')

In [ ]:
# SQL Statement
spark.sql('''
          select min(tpep_pickup_datetime), max(tpep_dropoff_datetime)
          from tbl_raw_yellow_taxis_clean_s1
          ''').show(n=5)

---
### If we want to write the output, for example partitioned by date

In [ ]:
# Create new partition key
df_sink = df_clean_s1.withColumn("p_date",to_date(col('tpep_pickup_datetime')))

In [ ]:
# Write to local storage, if not done already:
df_sink.write.partitionBy("p_date").mode("overwrite").parquet("datasets/yellow_taxis_daily/")

In [ ]:
# Stop the session
spark.stop()